<a href="https://colab.research.google.com/github/oddrationale/AdventOfCode2023Python/blob/main/Day02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<article class="day-desc"><h2>--- Day 2: Cube Conundrum ---</h2><p>You're launched high into the atmosphere! The apex of your trajectory just barely reaches the surface of a large island floating in the sky. You gently land in a fluffy pile of leaves. It's quite cold, but you don't see much snow. An Elf runs over to greet you.</p>
<p>The Elf explains that you've arrived at <em>Snow Island</em> and apologizes for the lack of snow. He'll be happy to explain the situation, but it's a bit of a walk, so you have some time. They don't get many visitors up here; <span title="No, the Elf's name is not 'WOPR'. It's Joshua.">would you like to play a game</span> in the meantime?</p>
<p>As you walk, the Elf shows you a small bag and some cubes which are either red, green, or blue. Each time you play this game, he will hide a secret number of cubes of each color in the bag, and your goal is to figure out information about the number of cubes.</p>
<p>To get information, once a bag has been loaded with cubes, the Elf will reach into the bag, grab a handful of random cubes, show them to you, and then put them back in the bag. He'll do this a few times per game.</p>
<p>You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the <code>11</code> in <code>Game 11: ...</code>) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like <code>3 red, 5 green, 4 blue</code>).</p>
<p>For example, the record of a few games might look like this:</p>
<pre><code>Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
</code></pre>
<p>In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.</p>
<p>The Elf would first like to know which games would have been possible if the bag contained <em>only 12 red cubes, 13 green cubes, and 14 blue cubes</em>?</p>
<p>In the example above, games 1, 2, and 5 would have been <em>possible</em> if the bag had been loaded with that configuration. However, game 3 would have been <em>impossible</em> because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been <em>impossible</em> because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get <code><em>8</em></code>.</p>
<p>Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. <em>What is the sum of the IDs of those games?</em></p>
</article>

In [ ]:
from google.colab import drive
drive.mount(r"/content/drive")

input_file = r"/content/drive/MyDrive/Colab Notebooks/AoC2023/input/02.txt"
with open(input_file, "r") as file:
    input = file.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pydantic import BaseModel

In [ ]:
class Set(BaseModel):
    """A class to represent a set of colored cubes in a game.

    Attributes:
        red (int): The number of red cubes in the set. Defaults to 0.
        green (int): The number of green cubes in the set. Defaults to 0.
        blue (int): The number of blue cubes in the set. Defaults to 0.
    """

    red: int = 0
    green: int = 0
    blue: int = 0


class Game(BaseModel):
    """A class to represent a game with multiple sets of colored cubes.

    Attributes:
        game_id (int): The unique identifier of the game.
        sets (list[Set]): The list of sets of colored cubes in the game.
    """

    game_id: int
    sets: list[Set]

    @classmethod
    def from_string(cls, string: str) -> "Game":
        """Creates a game object from a string representation.

        Args:
            string (str): The string representation of the game. It should be in the format "Game id: set1; set2; ...",
            where id is an integer and each set is a comma-separated list of color and number pairs, such as "red 3,
            green 2, blue 1".

        Returns:
            Game: The game object corresponding to the string.

        Raises:
            ValueError: If the string is not in the expected format or contains invalid values.
        """
        # Split the string by ": " to get the game id and the sets
        game_str, sets_str = string.split(": ")
        # Extract the game id from the game string
        game_id = int(game_str.split(" ")[-1])
        # Split the sets string by "; " to get the individual sets
        sets = [
            # For each set, create a dictionary of color and number pairs
            dict([(k, v) for v, k in [color.split() for color in s.split(", ")]])
            # Convert the numbers to integers
            for s in sets_str.split("; ")
        ]
        # Use the parse_obj method of the BaseModel class to create the game object from the dictionary
        return cls.parse_obj(
            {
                "game_id": game_id,
                "sets": sets,
            }
        )

    def is_possible(self, red: int, green: int, blue: int) -> bool:
        """Checks if it is possible to complete the game with the given number of colored cubes.

        Args:
            red (int): The number of red cubes available.
            green (int): The number of green cubes available.
            blue (int): The number of blue cubes available.

        Returns:
            bool: True if it is possible to complete the game, False otherwise.
        """
        # Loop through the sets of colored cubes in the game
        for s in self.sets:
            # If any set requires more cubes of a certain color than the available number, return False
            if s.red > red or s.green > green or s.blue > blue:
                return False
        # If none of the sets requires more cubes of a certain color than the available number, return True
        return True

    # for part 2
    def power(self) -> int:
        """Calculates the power of the game based on the maximum number of cubes of each color in the sets.

        Returns:
            int: The power of the game. The formula is the product of the maximum number of red, green, and blue cubes
            in the sets.
        """
        # Find the maximum number of red cubes in the sets
        max_red = max([s.red for s in self.sets])
        # Find the maximum number of green cubes in the sets
        max_green = max([s.green for s in self.sets])
        # Find the maximum number of blue cubes in the sets
        max_blue = max([s.blue for s in self.sets])
        # Return the product of the maximum numbers
        return max_red * max_green * max_blue


In [ ]:
assert Game.from_string(
    "Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green"
).is_possible(12, 13, 14) == True

assert Game.from_string(
    "Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue"
).is_possible(12, 13, 14) == True

assert Game.from_string(
    "Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red"
).is_possible(12, 13, 14) == False

assert Game.from_string(
    "Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red"
).is_possible(12, 13, 14) == False

assert Game.from_string(
    "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"
).is_possible(12, 13, 14) == True

In [ ]:
games = [Game.from_string(line) for line in input.split("\n") if line]

In [ ]:
sum([game.game_id for game in games if game.is_possible(12, 13, 14)])

2447

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Elf says they've stopped producing snow because they aren't getting any <em>water</em>! He isn't sure why the water stopped; however, he can show you how to get to the water source to check it out for yourself. It's just up ahead!</p>
<p>As you continue your walk, the Elf poses a second question: in each game you played, what is the <em>fewest number of cubes of each color</em> that could have been in the bag to make the game possible?</p>
<p>Again consider the example games from earlier:</p>
<pre><code>Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
</code></pre>
<ul>
<li>In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.</li>
<li>Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.<li>
<li>Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.</li>
<li>Game 4 required at least 14 red, 3 green, and 15 blue cubes.</li>
<li>Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.</li>
</ul>
<p>The <em>power</em> of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is <code>48</code>. In games 2-5 it was <code>12</code>, <code>1560</code>, <code>630</code>, and <code>36</code>, respectively. Adding up these five powers produces the sum <code><em>2286</em></code>.</p>
<p>For each game, find the minimum set of cubes that must have been present. <em>What is the sum of the power of these sets?</em></p>
</article>

In [ ]:
assert Game.from_string("Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green").power() == 48
assert Game.from_string("Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue").power() == 12
assert Game.from_string("Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red").power() == 1560
assert Game.from_string("Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red").power() == 630
assert Game.from_string("Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green").power() == 36

In [ ]:
sum([game.power() for game in games])

56322